setup: open trades database and table epex_12_20_12_13

# FlexPower Interview tasks -- Testbook

## Task 1 Minimal reporting tool

### set up and data import

In [68]:
#PYTHON 3.9.6

In [16]:
import sqlite3
import pandas as pd

In [ ]:
con = sqlite3.connect("trades.sqlite")
trades = pd.read_sql_query("SELECT * from epex_12_20_12_13", con)
#TODO: check this is reading all of them in
print(trades.head())
con.close()
print(trades.size, trades.shape)

        id  quantity  price  side    strategy
0  trade_1         5   20.0   buy  strategy_1
1  trade_2        17   10.0  sell  strategy_1
2  trade_3        15    8.0   buy  strategy_1
3  trade_4        13   35.0  sell  strategy_2
4  trade_5        23   11.0  sell  strategy_2
30 (6, 5)


In [33]:
print(trades)

        id  quantity  price  side    strategy
0  trade_1         5   20.0   buy  strategy_1
1  trade_2        17   10.0  sell  strategy_1
2  trade_3        15    8.0   buy  strategy_1
3  trade_4        13   35.0  sell  strategy_2
4  trade_5        23   11.0  sell  strategy_2
5  trade_6         8   20.0   buy  strategy_2


In [57]:
#TODO: quick data cleaner making sure evth is in the right type

### Task 1.1
Write a function that computes the total buy volume for flex power, another that computes the total sell volume.

In [22]:

def compute_total_buy_volume(tradetable) -> float:
    total_buy = tradetable[tradetable.side == 'buy'].quantity.sum()
    return total_buy

def compute_total_sell_volume(tradetable) -> float:
    total_sell = tradetable[tradetable.side == 'sell'].quantity.sum()
    return total_sell

In [23]:
compute_total_buy_volume(trades)
compute_total_sell_volume(trades)

np.int64(53)

In [ ]:
#TODO: change to dataframe.filter for efficiency (in case tested on a bigger database)

### Task 1.2
PnL calculation for each strategy; defn sum of incomes, with selling +ve and buying -ve

In [24]:
print(trades.strategy.unique())

['strategy_1' 'strategy_2']


In [65]:
def compute_pnl(strategy_id: str, tradetable) -> float:
    if strategy_id in tradetable.strategy.unique():
        this_strategy_trades_sell = tradetable[(tradetable.strategy == strategy_id) & (tradetable.side == 'sell')]
        this_strategy_trades_buy = tradetable[(tradetable.strategy == strategy_id) & (tradetable.side == 'buy')]
        sales = this_strategy_trades_sell.quantity * this_strategy_trades_sell.price
        sales_value = sales.sum()
        buys = this_strategy_trades_buy.quantity * this_strategy_trades_buy.price
        buys_value = buys.sum()
        income = sales_value - buys_value
    else:
        income = 0
    return income
#make this smarter anad use the buy sell volume funcs from above?
#use .dot explicityly rather than *? check know behaviour of this?
#change to not do memory allocation (change to a lambda function? rather than data slicing?

In [67]:
compute_pnl('strategy_2',trades)

np.float64(548.0)

### Task 1.3
expose compute_pnl as the entrypoint of a web application

import requests as rq

class Api:
    def compute_pnl(strategy_id: str, tradetable) -> float:
        if strategy_id in tradetable.strategy.unique():
            this_strategy_trades_sell = tradetable[(tradetable.strategy == strategy_id) & (tradetable.side == 'sell')]
            this_strategy_trades_buy = tradetable[(tradetable.strategy == strategy_id) & (tradetable.side == 'buy')]
            sales = this_strategy_trades_sell.quantity * this_strategy_trades_sell.price
            sales_value = sales.sum()
            buys = this_strategy_trades_buy.quantity * this_strategy_trades_buy.price
            buys_value = buys.sum()
            income = sales_value - buys_value
        else:
            income = 0
        return income

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/pnl/<strategy_id>")
def compute_pnl(strategy_id: str, tradetable) -> float:
        if strategy_id in tradetable.strategy.unique():
            this_strategy_trades_sell = tradetable[(tradetable.strategy == strategy_id) & (tradetable.side == 'sell')]
            this_strategy_trades_buy = tradetable[(tradetable.strategy == strategy_id) & (tradetable.side == 'buy')]
            sales = this_strategy_trades_sell.quantity * this_strategy_trades_sell.price
            sales_value = sales.sum()
            buys = this_strategy_trades_buy.quantity * this_strategy_trades_buy.price
            buys_value = buys.sum()
            income = sales_value - buys_value
        else:
            income = 0
        return jsonify(income),200

can't figure out how to do this yet; move on and check back in later

## Task 2

### Task 2.1